# Bia-bob
This notebook is used to generate a python-file that serves as interoperability connector to [bia-bob](https://github.com/haesleinhuepf/bia-bob). You need an OpenAI account to run this notebook. Note: It uses chatGPT in a loop and executing this notebook costs some cents.

In [1]:
import pyclesperanto_prototype as cle
import openai

In [2]:
def prompt(message:str, model="gpt-3.5-turbo"):
    """A prompt helper function that sends a message to openAI
    and returns only the text response.
    """
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": message}]
    )
    return response.choices[0].message.content

In [3]:
def get_function_signature(func):
    import inspect
    
    function_object = func
    signature_object = inspect.signature(function_object)
    signature_string = "cle." + func.__name__ + str(signature_object)
    signature_string = signature_string.replace("Union[numpy.ndarray, pyclesperanto_prototype._tier0._pycl.OCLArray, pyopencl._cl.Image, pyclesperanto_prototype._tier0._pycl._OCLImage]", "ndarray")
    return signature_string

all_descriptions = []
for name, func in cle.operations(['bia-bob-suggestion']).items():
    
    description = prompt(f"""Shorten the following docstring into a single bullet point:

    ```
    {func.__doc__}
    ```
    
    Shorten it to a single bullet point explaining what the function does without mentioning the function name and without phrases such as `use this function`.

    Examples:

    * compute the absolute value of every individual pixel in an image
    cle.absolute(source: ndarray, destination: ndarray = None) -> ndarray

    * determine the absolute difference pixel by pixel between two images
    cle.absolute_difference(source1: ndarray, source2: ndarray, destination: ndarray = None) -> ndarray
    
    * add a scalar value to all pixels of an image
    cle.add_image_and_scalar(source: ndarray, destination: ndarray = None, scalar: float = 1) -> ndarray
    
    """).replace("If you want to ", "* ")
    description = description + "\n" + get_function_signature(func) + "\n\n"
    print(description)
    
    all_descriptions.append(description)

print(all_descriptions[:1000])

- Computes the absolute value of every individual pixel in a given image.
cle.absolute(source: ndarray, destination: ndarray = None) -> ndarray


- Determines the absolute difference pixel by pixel between two images.
cle.absolute_difference(source1: ndarray, source2: ndarray, destination: ndarray = None) -> ndarray


- Replace label map with the average distance to the n closest neighboring labels, given a label map, distance map, and n.
cle.average_distance_of_n_nearest_neighbors_map(labels: ndarray, distance_map: ndarray = None, n: int = 1) -> ndarray


- Replaces every label in the label map with the average distance to the n closest neighboring labels
cle.average_distance_of_n_nearest_neighbors_map(labels: ndarray, distance_map: ndarray = None, n: int = 1) -> ndarray


- Replaces labels in a label map with the average distance to neighboring labels.
cle.average_neighbor_distance_map(labels: ndarray, distance_map: ndarray = None) -> ndarray


- Compute a binary image from two input

In [4]:
descriptions = []
for bullet_point in all_descriptions:
    if bullet_point[0:2] in ['- ', '* ']:
        bullet_point = bullet_point[2:]

    descriptions.append("* " + bullet_point)


In [5]:
text = "".join(descriptions).replace('"', "'")
text = "    " + text.replace("\n", "\n    ")
text = f'''
def list_bia_bob_plugins():
    """List of function hints for bia_bob"""
    return """{text}"""
'''

with open("../../pyclesperanto_prototype/_bia_bob_plugins.py", 'w') as file:
    file.write(text)